In [1]:
import pandas as pd

## DATA LOADING, CLEANING & INTEGRATION

In [2]:
# Load datasets

orders = pd.read_csv("Orders.csv", encoding="latin-1", sep=";", decimal =",")
leadtime = pd.read_csv("LeadtimeService.csv", encoding="latin-1", sep=";")
airports = pd.read_csv("airports.csv", encoding="latin-1", sep=",", decimal =",")


In [3]:
orders

,Type,Direction,DSV-CW Ref.,Goods Description,Pcs,UQ,Gwgt,UQ.1,Cwgt,UQ.2,...,Service Level,DGR,Real Origin Airport,Real Destination Airport,CTRY \nORIGIN-DESTINATION,AIRPORT \nORIGIN-DESTINATION,ZONE,Tiempo (h),Request weekday,Request Delivery Day
0,ROA,Export,SBCN0269394,ENGINE 645612 + STAND,1,PCE,9800.00,KG,9800.000,KG,...,ROV,Non HAZ,BCN,MAD,ES-ES,BCN - MAD,Z1,47:19:00,mi,vi
1,ROA,Export,SBCN0266096,ENGINE P800230 + STAND SN341,2,PCE,8025.00,KG,8025.000,KG,...,ROV,Non HAZ,BCN,MAD,ES-ES,BCN - MAD,Z1,11:31:00,ju,vi
2,ROA,Export,SBCN0257348,CIVIL AIRCRAFT ENGINE,1,CRT,7200.00,KG,9318.681,KG,...,AOV,Non HAZ,BCN,LHR,ES-GB,BCN - LHR,Z3,112:10:00,ju,ma
3,ROA,Export,SBCN0257611,CIVIL AIRCRAFT ENGINE 849411,1,CRT,7200.00,KG,9773.227,KG,...,AOV,Non HAZ,BCN,LHR,ES-GB,BCN - LHR,Z3,111:42:00,ju,ma
4,ROA,Import,SBCN0271037,AC PARTS / ENGINE,2,PLT,7200.00,KG,7200.000,KG,...,AOV,Non HAZ,MAD,BCN,ES-ES,MAD - BCN,Z1,109:56:00,mi,lu
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3690,DSx,Import,SBCN0267373,AC PARTS / ROUTINE,1,PKG,0.10,KG,0.330,KG,...,ROV,Non HAZ,TLS,BCN,FR-ES,TLS - BCN,Z2,50:38:00,ma,ju
3691,AIR,Export,SBCN0269726,CIVIL AIRCRAFT PARTS,1,BOX,0.10,KG,0.330,KG,...,AOV,Non HAZ,BCN,BEG,ES-RS,BCN - BEG,Z3,37:22:00,lu,mi
3692,AIR,Import,SLHR0368471,CIVIL AIRCRAFT PARTS - FLAP MOUNT,1,BOX,0.10,KG,2.000,KG,...,AOV,Non HAZ,LHR,BCN,GB-ES,LHR - BCN,Z3,21:58:00,lu,ma
3693,DSx,Import,SBCN0274152,AC PARTS - ROUTINE,1,PKG,0.03,KG,3.000,KG,...,ROV,Non HAZ,PRG,BCN,CZ-ES,PRG - BCN,Z2,48:30:00,mi,vi


In [4]:
# Select relevant columns for analysis from orders dataset

cols_keep = [
    "Type",
    "Direction",
    "DSV-CW Ref.",
    "Goods Description",
    "Pcs",
    "Gwgt",
    "Cwgt",
    "NOTIFICATION date & Time",
    "ACTUAL Delivery & Time",
    "Service Level",
    "DGR",
    "Real Origin Airport",
    "Real Destination Airport",
    "ZONE"
]

df_analysis = orders[cols_keep]

In [5]:
df_analysis["Service Level"].value_counts()

Service Level
ROV                                    2099
AOV                                    1233
CRV                                     330
ERROR en Service Level (columna AK)      33
Name: count, dtype: int64

In [6]:
#drop rows with service level: "ERROR en Service Level (columna AK)"

df_analysis = df_analysis[df_analysis["Service Level"] != "ERROR en Service Level (columna AK)"]

In [7]:
df_analysis["Service Level"].value_counts()

Service Level
ROV    2099
AOV    1233
CRV     330
Name: count, dtype: int64

In [8]:
# Service Level mapping used in the analysis
service_level_mapping = {
    "ROV": "Normal",
    "CRV": "Urgent",
    "AOV": "Critical"
}

df_analysis["Service Type"] = (
    df_analysis["Service Level"]
    .map(service_level_mapping)
)

df_analysis[["Service Level", "Service Type"]].drop_duplicates()

,Service Level,Service Type
0,ROV,Normal
2,AOV,Critical
19,CRV,Urgent


In [9]:
df_analysis["ZONE"].value_counts()

ZONE
Z2    2080
Z4     719
Z3     662
Z5     139
Z6      41
Z1      21
Name: count, dtype: int64

In [10]:
leadtime

,Zone,Zone description,SLA,Leadtime
0,Z6,Pending,ROV,100
1,Z6,Pending,CRV,100
2,Z6,Pending,AOV,100
3,Z5,Other,ROV,72
4,Z5,Other,CRV,48
5,Z5,Other,AOV,24
6,Z4,N.A.,ROV,72
7,Z4,N.A.,CRV,48
8,Z4,N.A.,AOV,24
9,Z3,EUR,ROV,72


In [11]:
#Drop spaces in column names
df_analysis.columns = df_analysis.columns.str.strip()
leadtime.columns = leadtime.columns.str.strip()

# Clean string columns to ensure proper merging
df_analysis["ZONE"] = df_analysis["ZONE"].astype(str).str.strip()
df_analysis["Service Level"] = df_analysis["Service Level"].astype(str).str.strip()

leadtime["Zone"] = leadtime["Zone"].astype(str).str.strip()
leadtime["SLA"] = leadtime["SLA"].astype(str).str.strip()

# Merge lead time information into the main dataframe

df_analysis = df_analysis.merge(
    leadtime[["Zone", "SLA", "Leadtime"]],
    how="left",
    left_on=["ZONE", "Service Level"],
    right_on=["Zone", "SLA"]
)

# Limpiar columnas auxiliares del join
df_analysis.drop(columns=["Zone", "SLA"], inplace=True)


In [12]:
df_analysis

,Type,Direction,DSV-CW Ref.,Goods Description,Pcs,Gwgt,Cwgt,NOTIFICATION date & Time,ACTUAL Delivery & Time,Service Level,DGR,Real Origin Airport,Real Destination Airport,ZONE,Service Type,Leadtime
0,ROA,Export,SBCN0269394,ENGINE 645612 + STAND,1,9800.00,9800.000,07/05/2025 15:41,09/05/2025 15:00,ROV,Non HAZ,BCN,MAD,Z1,Normal,24
1,ROA,Export,SBCN0266096,ENGINE P800230 + STAND SN341,2,8025.00,8025.000,03/04/2025 12:29,04/04/2025 0:00,ROV,Non HAZ,BCN,MAD,Z1,Normal,24
2,ROA,Export,SBCN0257348,CIVIL AIRCRAFT ENGINE,1,7200.00,9318.681,16/01/2025 18:30,21/01/2025 10:40,AOV,Non HAZ,BCN,LHR,Z3,Critical,24
3,ROA,Export,SBCN0257611,CIVIL AIRCRAFT ENGINE 849411,1,7200.00,9773.227,16/01/2025 16:18,21/01/2025 8:00,AOV,Non HAZ,BCN,LHR,Z3,Critical,24
4,ROA,Import,SBCN0271037,AC PARTS / ENGINE,2,7200.00,7200.000,21/05/2025 11:00,26/05/2025 0:56,AOV,Non HAZ,MAD,BCN,Z1,Critical,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,DSx,Import,SBCN0267373,AC PARTS / ROUTINE,1,0.10,0.330,15/04/2025 11:00,17/04/2025 13:38,ROV,Non HAZ,TLS,BCN,Z2,Normal,36
3658,AIR,Export,SBCN0269726,CIVIL AIRCRAFT PARTS,1,0.10,0.330,12/05/2025 10:38,14/05/2025 0:00,AOV,Non HAZ,BCN,BEG,Z3,Critical,24
3659,AIR,Import,SLHR0368471,CIVIL AIRCRAFT PARTS - FLAP MOUNT,1,0.10,2.000,19/05/2025 19:32,20/05/2025 17:30,AOV,Non HAZ,LHR,BCN,Z3,Critical,24
3660,DSx,Import,SBCN0274152,AC PARTS - ROUTINE,1,0.03,3.000,18/06/2025 11:00,20/06/2025 11:30,ROV,Non HAZ,PRG,BCN,Z2,Normal,36


In [13]:
airports

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,icao_code,iata_code,gps_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total RF Heliport,40.070985,-74.933689,11.0,NaN,US,US-PA,Bensalem,no,NaN,NaN,K00A,00A,https://www.penndot.pa.gov/TravelInPA/airports...,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,NaN,NaN,00AA,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.947733,-151.692524,450.0,NaN,US,US-AK,Anchor Point,no,NaN,NaN,00AK,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.86479949951172,-86.77030181884766,820.0,NaN,US,US-AL,Harvest,no,NaN,NaN,00AL,00AL,NaN,NaN,NaN
4,506791,00AN,small_airport,Katmai Lodge Airport,59.093287,-156.456699,80.0,NaN,US,US-AK,King Salmon,no,NaN,NaN,00AN,00AN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84485,32753,ZYYY,medium_airport,Shenyang Dongta Airport,41.784354,123.496308,157.0,AS,CN,CN-21,"Dadong, Shenyang",no,ZYYY,NaN,ZYYY,NaN,NaN,NaN,"ä¸å¡æºåº, SHE"
84486,46378,ZZ-0001,heliport,Sealand Helipad,51.894444,1.4825,40.0,EU,GB,GB-ENG,Sealand,no,NaN,NaN,NaN,NaN,http://www.sealandgov.org/,https://en.wikipedia.org/wiki/Principality_of_...,Roughs Tower Helipad
84487,307326,ZZ-0002,small_airport,Glorioso Islands Airstrip,-11.584277777799999,47.296388888900005,11.0,AF,TF,TF-U-A,Grande Glorieuse,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84488,346788,ZZ-0003,small_airport,Fainting Goat Airport,32.110587,-97.356312,690.0,NaN,US,US-TX,Blum,no,NaN,NaN,87TX,87TX,NaN,NaN,NaN


In [14]:
#Drop spaces in column names
airports.columns = airports.columns.str.strip()

# Clean string columns to ensure proper merging
df_analysis["Real Origin Airport"] = df_analysis["Real Origin Airport"].astype(str).str.strip()
df_analysis["Real Destination Airport"] = df_analysis["Real Destination Airport"].astype(str).str.strip()

airports["iata_code"] = airports["iata_code"].astype(str).str.strip()

# Merge information into the main dataframe for ORIGIN airport
df_analysis = df_analysis.merge(
    airports[["iata_code", "iso_country"]],
    how="left",
    left_on="Real Origin Airport",
    right_on="iata_code"
)

# Rename columns to avoid confusion
df_analysis.rename(
    columns={
        "iata_code": "iata_code_origin",
        "iso_country": "iso_country_origin"
    },
    inplace=True
)

# Merge information into the main dataframe for DESTINATION airport
df_analysis = df_analysis.merge(
    airports[["iata_code", "iso_country"]],
    how="left",
    left_on="Real Destination Airport",
    right_on="iata_code"
)

# Rename columns to avoid confusion
df_analysis.rename(
    columns={
        "iata_code": "iata_code_destination",
        "iso_country": "iso_country_destination"
    },
    inplace=True
)


In [15]:
# Final dataframe for analysis
df_analysis = df_analysis.drop(columns=["Real Origin Airport", "Real Destination Airport"])
df_analysis

,Type,Direction,DSV-CW Ref.,Goods Description,Pcs,Gwgt,Cwgt,NOTIFICATION date & Time,ACTUAL Delivery & Time,Service Level,DGR,ZONE,Service Type,Leadtime,iata_code_origin,iso_country_origin,iata_code_destination,iso_country_destination
0,ROA,Export,SBCN0269394,ENGINE 645612 + STAND,1,9800.00,9800.000,07/05/2025 15:41,09/05/2025 15:00,ROV,Non HAZ,Z1,Normal,24,BCN,ES,MAD,ES
1,ROA,Export,SBCN0266096,ENGINE P800230 + STAND SN341,2,8025.00,8025.000,03/04/2025 12:29,04/04/2025 0:00,ROV,Non HAZ,Z1,Normal,24,BCN,ES,MAD,ES
2,ROA,Export,SBCN0257348,CIVIL AIRCRAFT ENGINE,1,7200.00,9318.681,16/01/2025 18:30,21/01/2025 10:40,AOV,Non HAZ,Z3,Critical,24,BCN,ES,LHR,GB
3,ROA,Export,SBCN0257611,CIVIL AIRCRAFT ENGINE 849411,1,7200.00,9773.227,16/01/2025 16:18,21/01/2025 8:00,AOV,Non HAZ,Z3,Critical,24,BCN,ES,LHR,GB
4,ROA,Import,SBCN0271037,AC PARTS / ENGINE,2,7200.00,7200.000,21/05/2025 11:00,26/05/2025 0:56,AOV,Non HAZ,Z1,Critical,4,MAD,ES,BCN,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,DSx,Import,SBCN0267373,AC PARTS / ROUTINE,1,0.10,0.330,15/04/2025 11:00,17/04/2025 13:38,ROV,Non HAZ,Z2,Normal,36,TLS,FR,BCN,ES
3658,AIR,Export,SBCN0269726,CIVIL AIRCRAFT PARTS,1,0.10,0.330,12/05/2025 10:38,14/05/2025 0:00,AOV,Non HAZ,Z3,Critical,24,BCN,ES,BEG,RS
3659,AIR,Import,SLHR0368471,CIVIL AIRCRAFT PARTS - FLAP MOUNT,1,0.10,2.000,19/05/2025 19:32,20/05/2025 17:30,AOV,Non HAZ,Z3,Critical,24,LHR,GB,BCN,ES
3660,DSx,Import,SBCN0274152,AC PARTS - ROUTINE,1,0.03,3.000,18/06/2025 11:00,20/06/2025 11:30,ROV,Non HAZ,Z2,Normal,36,PRG,CZ,BCN,ES


## TARGET ENGINEERING (leadtime_real)

In [16]:
#Convert date columns to datetime format

notification_col = "NOTIFICATION date & Time"
delivery_col = "ACTUAL Delivery & Time"

df_analysis[notification_col] = pd.to_datetime(
    df_analysis[notification_col],
    errors="coerce",
    dayfirst=True
)

df_analysis[delivery_col] = pd.to_datetime(
    df_analysis[delivery_col],
    errors="coerce",
    dayfirst=True
)

print("\nMissing timestamps after datetime conversion:")
display(df_analysis[[notification_col, delivery_col]].isna().sum())


Missing timestamps after datetime conversion:


NOTIFICATION date & Time    0
ACTUAL Delivery & Time      1
dtype: int64

In [17]:
#Drop rows with missing timestamps
df_analysis = df_analysis.dropna(subset=[notification_col, delivery_col])

df_analysis

,Type,Direction,DSV-CW Ref.,Goods Description,Pcs,Gwgt,Cwgt,NOTIFICATION date & Time,ACTUAL Delivery & Time,Service Level,DGR,ZONE,Service Type,Leadtime,iata_code_origin,iso_country_origin,iata_code_destination,iso_country_destination
0,ROA,Export,SBCN0269394,ENGINE 645612 + STAND,1,9800.00,9800.000,2025-05-07 15:41:00,2025-05-09 15:00:00,ROV,Non HAZ,Z1,Normal,24,BCN,ES,MAD,ES
1,ROA,Export,SBCN0266096,ENGINE P800230 + STAND SN341,2,8025.00,8025.000,2025-04-03 12:29:00,2025-04-04 00:00:00,ROV,Non HAZ,Z1,Normal,24,BCN,ES,MAD,ES
2,ROA,Export,SBCN0257348,CIVIL AIRCRAFT ENGINE,1,7200.00,9318.681,2025-01-16 18:30:00,2025-01-21 10:40:00,AOV,Non HAZ,Z3,Critical,24,BCN,ES,LHR,GB
3,ROA,Export,SBCN0257611,CIVIL AIRCRAFT ENGINE 849411,1,7200.00,9773.227,2025-01-16 16:18:00,2025-01-21 08:00:00,AOV,Non HAZ,Z3,Critical,24,BCN,ES,LHR,GB
4,ROA,Import,SBCN0271037,AC PARTS / ENGINE,2,7200.00,7200.000,2025-05-21 11:00:00,2025-05-26 00:56:00,AOV,Non HAZ,Z1,Critical,4,MAD,ES,BCN,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,DSx,Import,SBCN0267373,AC PARTS / ROUTINE,1,0.10,0.330,2025-04-15 11:00:00,2025-04-17 13:38:00,ROV,Non HAZ,Z2,Normal,36,TLS,FR,BCN,ES
3658,AIR,Export,SBCN0269726,CIVIL AIRCRAFT PARTS,1,0.10,0.330,2025-05-12 10:38:00,2025-05-14 00:00:00,AOV,Non HAZ,Z3,Critical,24,BCN,ES,BEG,RS
3659,AIR,Import,SLHR0368471,CIVIL AIRCRAFT PARTS - FLAP MOUNT,1,0.10,2.000,2025-05-19 19:32:00,2025-05-20 17:30:00,AOV,Non HAZ,Z3,Critical,24,LHR,GB,BCN,ES
3660,DSx,Import,SBCN0274152,AC PARTS - ROUTINE,1,0.03,3.000,2025-06-18 11:00:00,2025-06-20 11:30:00,ROV,Non HAZ,Z2,Normal,36,PRG,CZ,BCN,ES


In [18]:
#Creating leadtime real in hours

df_analysis["leadtime_real_hours"] = (
    (df_analysis[delivery_col] - df_analysis[notification_col])
    .dt.total_seconds() / 3600
)

df_analysis[["leadtime_real_hours", notification_col, delivery_col]]


/var/folders/mg/wjsd2dzj6zgdzk_gsdz19rdc0000gn/T/ipykernel_80278/2456794107.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis["leadtime_real_hours"] = (


,leadtime_real_hours,NOTIFICATION date & Time,ACTUAL Delivery & Time
0,47.316667,2025-05-07 15:41:00,2025-05-09 15:00:00
1,11.516667,2025-04-03 12:29:00,2025-04-04 00:00:00
2,112.166667,2025-01-16 18:30:00,2025-01-21 10:40:00
3,111.700000,2025-01-16 16:18:00,2025-01-21 08:00:00
4,109.933333,2025-05-21 11:00:00,2025-05-26 00:56:00
...,...,...,...
3657,50.633333,2025-04-15 11:00:00,2025-04-17 13:38:00
3658,37.366667,2025-05-12 10:38:00,2025-05-14 00:00:00
3659,21.966667,2025-05-19 19:32:00,2025-05-20 17:30:00
3660,48.500000,2025-06-18 11:00:00,2025-06-20 11:30:00


In [19]:
# Negative lead times check because of data errors
print("Negatives:", (df_analysis["leadtime_real_hours"] < 0).sum())


Negatives: 27


In [20]:
# Dealing with negative and 0 lead times

negatives_df = df_analysis[df_analysis["leadtime_real_hours"] <= 0]

print(f"Negative lead times: {len(negatives_df)}")

display(
    negatives_df[
        [
            "NOTIFICATION date & Time",
            "ACTUAL Delivery & Time",
            "leadtime_real_hours",
            "ZONE",
            "Service Level",
            "Service Type"
        ]
    ].sort_values("leadtime_real_hours")
)

Negative lead times: 31


,NOTIFICATION date & Time,ACTUAL Delivery & Time,leadtime_real_hours,ZONE,Service Level,Service Type
1605,2025-05-03 12:30:00,2022-06-05 14:43:00,-25509.783333,Z2,AOV,Critical
1748,2025-01-28 09:13:00,2024-01-29 09:15:00,-8759.966667,Z2,CRV,Urgent
620,2025-01-06 15:55:00,2024-01-07 18:22:00,-8757.550000,Z2,AOV,Critical
3219,2025-01-06 14:00:00,2024-01-07 18:22:00,-8755.633333,Z2,AOV,Critical
3098,2025-01-05 09:01:00,2024-01-06 20:35:00,-8748.433333,Z4,AOV,Critical
7,2025-01-30 08:03:00,2024-02-01 07:50:00,-8736.216667,Z3,AOV,Critical
3101,2025-04-04 09:40:00,2024-04-08 16:00:00,-8657.666667,Z2,ROV,Normal
280,2025-09-06 10:00:00,2025-06-12 16:00:00,-2058.000000,Z2,ROV,Normal
2017,2025-01-29 18:24:00,2024-12-30 11:16:00,-727.133333,Z2,ROV,Normal
2317,2025-06-19 08:19:00,2025-06-18 15:00:00,-17.316667,Z2,CRV,Urgent


In [21]:
#Flagging invalid lead times

df_analysis["invalid_leadtime_flag"] = (df_analysis["leadtime_real_hours"] <= 0).astype(int)

df_analysis["invalid_leadtime_flag"].value_counts()

/var/folders/mg/wjsd2dzj6zgdzk_gsdz19rdc0000gn/T/ipykernel_80278/3267888158.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis["invalid_leadtime_flag"] = (df_analysis["leadtime_real_hours"] <= 0).astype(int)


invalid_leadtime_flag
0    3630
1      31
Name: count, dtype: int64

In [22]:
df_analysis = df_analysis[df_analysis["leadtime_real_hours"] > 0].copy()

#Statistics summary
print(df_analysis["leadtime_real_hours"].describe())


count    3630.000000
mean       83.871933
std        73.270683
min         0.116667
25%        27.216667
50%        65.616667
75%       119.437500
max       670.783333
Name: leadtime_real_hours, dtype: float64


In [23]:
#Creating a target with 99th percentile lead time for avoid outliers

leadtime_99th_percentile = df_analysis["leadtime_real_hours"].quantile(0.99)

df_analysis["leadtime_ml_hours"] = df_analysis["leadtime_real_hours"].clip(upper=leadtime_99th_percentile)

#Statistics summary
print(df_analysis["leadtime_ml_hours"].describe())

count    3630.000000
mean       83.088086
std        69.492946
min         0.116667
25%        27.216667
50%        65.616667
75%       119.437500
max       337.094667
Name: leadtime_ml_hours, dtype: float64


In [24]:
# leadtime_real_hours: real operational lead time (EDA)
# leadtime_ml_hours: capped target used for ML training with 99th percentile

In [26]:
#Calculating on_time comparing leadtime_real_hours with SLA
df_analysis["on_time"] = (df_analysis["leadtime_real_hours"] <= df_analysis["Leadtime"]).astype(int)

In [ ]:
df_analysis

## Exploratory Data


In [79]:

# Ranking SLA service levels (1=cheapest, 3=most expensive)
sla_rank = {
    "Normal": 1,
    "Urgent": 2,
    "Critical": 3
}

# Create a lookup dictionary for SLA targets by ZONE + Service Type
sla_targets_lookup = (
    df_analysis
    .groupby(["ZONE", "Service Type"])["Leadtime"]
    .first()
    .to_dict()
)

# Function to determine minimum required SLA based on actual delivery time and zone
def calculate_minimum_required_sla(row):
    """
    Determines the cheapest SLA level that would have met the delivery time
    for the specific ZONE of this order.
    """
    actual_leadtime = row["leadtime_real_hours"]
    zone = row["ZONE"]
    
    # Check from cheapest to most expensive for this specific zone
    service_levels = ["Normal", "Urgent", "Critical"]
    
    for service_type in service_levels:
        target_key = (zone, service_type)
        if target_key in sla_targets_lookup:
            target_hours = sla_targets_lookup[target_key]
            if actual_leadtime <= target_hours:
                return service_type
    
    return "Out of SLA"

# Apply the function
df_analysis["sla_required"] = df_analysis.apply(calculate_minimum_required_sla, axis=1)

# Recommend SLA based on delivery performance
def recommend_sla(row):
    if row["sla_required"] == "Out of SLA":
        return "Normal"  # Late delivery gets minimum rate
    elif row["sla_required"] != row["Service Type"]:
        return row["sla_required"]  # Could have used cheaper SLA
    else:
        return row["Service Type"]  # Keep contracted level

df_analysis["sla_recommended"] = df_analysis.apply(recommend_sla, axis=1)

# Map to ranks for comparison
df_analysis["sla_contracted_rank"] = df_analysis["Service Type"].map(sla_rank)
df_analysis["sla_recommended_rank"] = df_analysis["sla_recommended"].map(sla_rank)

# Calculate levels to downgrade (how many ranks cheaper)
df_analysis["levels_to_downgrade"] = (
    df_analysis["sla_contracted_rank"] - df_analysis["sla_recommended_rank"]
)

# Downgrade is only possible if there are actual savings (levels_to_downgrade > 0)
df_analysis["downgrade_possible"] = (df_analysis["levels_to_downgrade"] > 0)

# Summary statistics
print("\n" + "="*60)
print("DOWNGRADE ANALYSIS SUMMARY")
print("="*60)
print(f"\nTotal orders analyzed: {len(df_analysis)}")
print(f"Orders with downgrade opportunity: {df_analysis['downgrade_possible'].sum()}")
print(f"  - With savings (levels > 0): {(df_analysis['levels_to_downgrade'] > 0).sum()}")
print(f"  - No savings (levels = 0): {(df_analysis['levels_to_downgrade'] == 0).sum()}")
print(f"Downgrade rate: {df_analysis['downgrade_possible'].mean():.1%}")

print("\nBreakdown by levels_to_downgrade:")
print(df_analysis['levels_to_downgrade'].value_counts().sort_index())

print("\nDowngrade opportunities by contracted Service Type:")
downgrade_by_service = df_analysis.groupby("Service Type").agg({
    'downgrade_possible': ['sum', 'mean'],
    'levels_to_downgrade': 'mean'
})
print(downgrade_by_service)




DOWNGRADE ANALYSIS SUMMARY

Total orders analyzed: 3630
Orders with downgrade opportunity: 1544
  - With savings (levels > 0): 1544
  - No savings (levels = 0): 2086
Downgrade rate: 42.5%

Breakdown by levels_to_downgrade:
levels_to_downgrade
0    2086
1     325
2    1219
Name: count, dtype: int64

Downgrade opportunities by contracted Service Type:
             downgrade_possible      levels_to_downgrade
                            sum mean                mean
Service Type                                            
Critical                   1219  1.0                 2.0
Normal                        0  0.0                 0.0
Urgent                      325  1.0                 1.0


In [80]:
# Show examples
print("\n" + "="*60)
print("EXAMPLES:")
print("="*60)

print("\n1. Downgrade possible (levels > 0):")
print(df_analysis[df_analysis["downgrade_possible"]][
    ["DSV-CW Ref.", "ZONE", "Service Type", "leadtime_real_hours", "Leadtime", 
     "sla_recommended", "levels_to_downgrade", "downgrade_possible"]
].head(3))

print("\n2. No downgrade (levels = 0, already at minimum or met SLA):")
print(df_analysis[df_analysis["levels_to_downgrade"] == 0][
    ["DSV-CW Ref.", "ZONE", "Service Type", "leadtime_real_hours", "Leadtime", 
     "sla_recommended", "levels_to_downgrade", "downgrade_possible"]
].head(3))

print("\n3. Specific order SBCN0260531:")
specific_order = df_analysis[df_analysis["DSV-CW Ref."] == "SBCN0260531"][
    ["DSV-CW Ref.", "ZONE", "Service Type", "leadtime_real_hours", "Leadtime", 
     "sla_recommended", "levels_to_downgrade", "downgrade_possible"]
]
if len(specific_order) > 0:
    print(specific_order)
else:
    print("Order not found in filtered dataset")


EXAMPLES:

1. Downgrade possible (levels > 0):
   DSV-CW Ref. ZONE Service Type  leadtime_real_hours  Leadtime  \
2  SBCN0257348   Z3     Critical           112.166667        24   
3  SBCN0257611   Z3     Critical           111.700000        24   
4  SBCN0271037   Z1     Critical           109.933333         4   

  sla_recommended  levels_to_downgrade  downgrade_possible  
2          Normal                    2                True  
3          Normal                    2                True  
4          Normal                    2                True  

2. No downgrade (levels = 0, already at minimum or met SLA):
    DSV-CW Ref. ZONE Service Type  leadtime_real_hours  Leadtime  \
0   SBCN0269394   Z1       Normal            47.316667        24   
1   SBCN0266096   Z1       Normal            11.516667        24   
10  SBCN0279058   Z1       Normal            74.333333        24   

   sla_recommended  levels_to_downgrade  downgrade_possible  
0           Normal                    0   

In [81]:
df_analysis

,Type,Direction,DSV-CW Ref.,Goods Description,Pcs,Gwgt,Cwgt,NOTIFICATION date & Time,ACTUAL Delivery & Time,Service Level,...,leadtime_ml_hours,on_time,sla_required,sla_contracted_rank,sla_required_rank,downgrade_possible,sla_recommended,sla_rank_difference,levels_to_downgrade,sla_recommended_rank
0,ROA,Export,SBCN0269394,ENGINE 645612 + STAND,1,9800.00,9800.000,2025-05-07 15:41:00,2025-05-09 15:00:00,ROV,...,47.316667,0,Out of SLA,1,NaN,False,Normal,NaN,0,1
1,ROA,Export,SBCN0266096,ENGINE P800230 + STAND SN341,2,8025.00,8025.000,2025-04-03 12:29:00,2025-04-04 00:00:00,ROV,...,11.516667,1,Normal,1,1.0,False,Normal,0.0,0,1
2,ROA,Export,SBCN0257348,CIVIL AIRCRAFT ENGINE,1,7200.00,9318.681,2025-01-16 18:30:00,2025-01-21 10:40:00,AOV,...,112.166667,0,Out of SLA,3,NaN,True,Normal,NaN,2,1
3,ROA,Export,SBCN0257611,CIVIL AIRCRAFT ENGINE 849411,1,7200.00,9773.227,2025-01-16 16:18:00,2025-01-21 08:00:00,AOV,...,111.700000,0,Out of SLA,3,NaN,True,Normal,NaN,2,1
4,ROA,Import,SBCN0271037,AC PARTS / ENGINE,2,7200.00,7200.000,2025-05-21 11:00:00,2025-05-26 00:56:00,AOV,...,109.933333,0,Out of SLA,3,NaN,True,Normal,NaN,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657,DSx,Import,SBCN0267373,AC PARTS / ROUTINE,1,0.10,0.330,2025-04-15 11:00:00,2025-04-17 13:38:00,ROV,...,50.633333,0,Out of SLA,1,NaN,False,Normal,NaN,0,1
3658,AIR,Export,SBCN0269726,CIVIL AIRCRAFT PARTS,1,0.10,0.330,2025-05-12 10:38:00,2025-05-14 00:00:00,AOV,...,37.366667,0,Normal,3,1.0,True,Normal,NaN,2,1
3659,AIR,Import,SLHR0368471,CIVIL AIRCRAFT PARTS - FLAP MOUNT,1,0.10,2.000,2025-05-19 19:32:00,2025-05-20 17:30:00,AOV,...,21.966667,1,Normal,3,1.0,True,Normal,2.0,2,1
3660,DSx,Import,SBCN0274152,AC PARTS - ROUTINE,1,0.03,3.000,2025-06-18 11:00:00,2025-06-20 11:30:00,ROV,...,48.500000,0,Out of SLA,1,NaN,False,Normal,NaN,0,1
